In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
%matplotlib notebook
import load_raw as Raw
import seaborn as sns
from scipy.special import expit as logit
from sklearn.model_selection import train_test_split
from scipy.special import expit as sigmoid # is more stable in case of overflows
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, \
recall_score, precision_score, accuracy_score, confusion_matrix
import math
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


# Prepare data

In [2]:
df_cleaned = pd.read_csv('/data/ortho/AllPatients.csv',  sep= ';')

# x is naar rechts
# y is omhoog
# z is rotatie????

In [3]:
# bereken symmetrie
df_cleaned['clavicula_x_dif'] = np.absolute(df_cleaned['clavicula_l_x'] - df_cleaned['clavicula_r_x'])
df_cleaned['clavicula_y_dif'] = np.absolute(df_cleaned['clavicula_l_y'] - df_cleaned['clavicula_r_y'])
df_cleaned['clavicula_z_dif'] = np.absolute(df_cleaned['clavicula_l_z'] - df_cleaned['clavicula_r_z'])

df_cleaned['scapula_x_dif'] = np.absolute(df_cleaned['scapula_l_x'] - df_cleaned['scapula_r_x'])
df_cleaned['scapula_y_dif'] = np.absolute(df_cleaned['scapula_l_y'] - df_cleaned['scapula_r_y'])
df_cleaned['scapula_z_dif'] = np.absolute(df_cleaned['scapula_l_z'] - df_cleaned['scapula_r_z'])

df_cleaned['humerus_x_dif'] = np.absolute(df_cleaned['humerus_l_x'] - df_cleaned['humerus_r_x'])
df_cleaned['humerus_y_dif'] = np.absolute(df_cleaned['humerus_l_y'] - df_cleaned['humerus_r_y'])
df_cleaned['humerus_z_dif'] = np.absolute(df_cleaned['humerus_l_z'] - df_cleaned['humerus_r_z'])

# hulp array, met alle parameters die voor de classifier gebruikt worden, je kan hier alles in doen wat je wilt
param = [ \
          'humerus_l_x', 'humerus_l_y', 'humerus_l_z', 'humerus_r_x', 'humerus_r_y', 'humerus_r_z', \
          'clavicula_l_x', 'clavicula_l_y', 'clavicula_l_z', 'clavicula_r_x', 'clavicula_r_y', 'clavicula_r_z', \
          'scapula_l_x', 'scapula_l_y', 'scapula_l_z', 'scapula_r_x', 'scapula_r_y', 'scapula_r_z', \
          'clavicula_x_dif','clavicula_y_dif','clavicula_z_dif', \
          'scapula_x_dif','scapula_y_dif','scapula_z_dif', \
          'humerus_x_dif', 'humerus_y_dif', 'humerus_z_dif'
         ]

df_cleaned['bias'] = 1

# split oorsprong kolom in onderdelen
x,y = df_cleaned['Oorsprong'].str.split(".").str #Oordprong word vertaald naar een string en wordt gesplits op de punt
df_cleaned['cat'],df_cleaned['pat'],df_cleaned['meting'],df_cleaned['oef'] = x.str.split("_").str #4 categorieen gemaakt obv file name
df_cleaned['cat'] = [ int(x[3:]) for x in df_cleaned['cat']] #voor elk 3+ element in de kolom wordt vertaald naar een int
df_cleaned['meting'] = [ int(x[6:]) for x in df_cleaned['meting']] 
df_cleaned['oef'] = [ int(x[3:]) for x in df_cleaned['oef']] 
df_cleaned['pat'] = [ int(x[3:]) for x in df_cleaned['pat']] 
#na deze regels te hebben uitgevoerd zijn er nieuwe categorieen met ints.

df_cleaned['pat'] = df_cleaned['cat']*1000+df_cleaned['pat'] #geef elke patient een uniek nummer

#maak boolean kolom per categorie
df_cleaned['c4'] = ['Cat4' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c3'] = ['Cat3' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c2'] = ['Cat2' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c1'] = ['Cat1' in vincent for vincent in df_cleaned['Oorsprong']]


# Define functions

In [4]:
# functie om plot van 2 van de variabelen uit de tabel df_cleaned te maken
def scatter(param1, param2):
    cat4 = df_cleaned.where(df_cleaned['c4'])
    cat3 = df_cleaned.where(df_cleaned['c3'])
    cat2 = df_cleaned.where(df_cleaned['c2'])
    cat1 = df_cleaned.where(df_cleaned['c1'])

    plt.plot(cat3[param1], cat3[param2], '.', color='red', markersize=2)
    plt.plot(cat1[param1], cat1[param2], '.', color='cyan', markersize=2)
    plt.plot(cat2[param1], cat2[param2], '.', color='green', markersize=2)
    plt.plot(cat4[param1], cat4[param2], '.', color='blue', markersize=2)
    plt.title(param1+'/'+param2)
    plt.ylabel(param1)
    plt.xlabel(param2);

def plot_decision_boundary(theta):
    ax = plt.gca()
    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 1),
                         np.arange(y_min, y_max, 1))

    X = np.matrix(np.vstack([np.ones(xx.shape[0] * xx.shape[1]), xx.ravel(), yy.ravel()])).T
    boundary = logit(X * theta)
    boundary = boundary.reshape(xx.shape)

    ax.contour(xx, yy,
           boundary,
           levels=[0.5])

def logit(z):
    return 1.0 / (1.0 + np.exp(-z))

def predict(X, theta):
    return h(X, theta) >= 0.5

def PlotData(Paramater1, Paramater2):
    # maak plotje van 2 variabelen
    # Scatter all data in different colors
    scatter(param[Paramater1-1],param[Paramater2-1])
    
    #Draw four decision boundaries
    for i in classes:
        theta =  Data[class_2b_found]['Model'].coef_.flatten()
        selectedtheta = np.array([theta[0], theta[Paramater1], theta[Paramater2]])
        selectedtheta = np.reshape(selectedtheta,(3,1))
        plot_decision_boundary(selectedtheta)
    plt.show()
    return

def CalculatePerc(cat, CountF1, CountF2, CountF3, CountF4):
    Count = [CountF1, CountF2, CountF3, CountF4]
    cat = int(cat)
    perc = 100*Count[cat-1]/np.nansum(Count)
    if(math.isnan(perc)):
        perc = 0
    return perc

def CategorieStringToNum(index):
    string = str(index)
    cat = string[0]
    return cat

# Learn models (x4)

In [5]:
# Doe hier alles wat niet in de loop gedaan hoeft te worden
    # Bereid de data opslag voor van de loop
classes = ['c1','c2','c3','c4']

Data = {}
for i in classes:
    Data[i] = {'Result': None, 
               'X_Train': None,
               'X_test': None, 
               'y_train': None, 
               'y_test': None, 
               'Model': None}
    # Einde data opslag voorbereiden



#hulpvariabele geeft aan welke categorie we willen onderscheiden
Xcolumns = ['bias']
Xcolumns.extend(param)

for class_2b_found in classes:
    classnum = int(class_2b_found[1])
    # <-- Begin model moken -->
    X = df_cleaned[Xcolumns]
    y = df_cleaned[class_2b_found]

    # Split Data into a test and train set
    Data[class_2b_found]['X_train'], \
    Data[class_2b_found]['X_test'], \
    Data[class_2b_found]['y_train'], \
    Data[class_2b_found]['y_test'] = train_test_split(X, y, test_size = 0.2, random_state=42)

    
    # De keuze voor een classifier is "LogisticRegression", vandaar dat lr= LogisticRegression
    Data[class_2b_found]['Model'] = MLPClassifier(hidden_layer_sizes=(50,40,50),solver='adam',random_state=21,tol=0.00001,learning_rate='adaptive',verbose=10)

    # Fit het model, met de train data (80 % )
    Data[class_2b_found]['Model'].fit(\
        Data[class_2b_found]['X_train'], Data[class_2b_found]['y_train'])
    # <-- Einde model maken -->
    
    
    # <-- Start percentage check -->
    # kolom met voorspelling toevoegen aan data
    # predict of de test data goed past bij deze LogisticRegression
    y_pred = Data[class_2b_found]['Model'].predict(Data[class_2b_found]['X_test'])
    
    PercentageDf = pd.DataFrame() # Create new dataframe
    PercentageDf['index'] = Data[class_2b_found]['X_test'].index # Get the index of the test data
    PercentageDf['predict'] = y_pred # Paste the True/False value in the predict column
    PercentageDf = PercentageDf.set_index('index') # zet de colom index als de index voor de join hierna
    PercentageDf = PercentageDf.join(df_cleaned[['pat', 'cat']]) # Join de pat en cat colom op de index
#     PercentageDf['ShouldBe'] = np.where(PercentageDf['cat'] == classnum, True, False)
#     PercentageDf['Correct2'] = np.where(PercentageDf['ShouldBe'] == PercentageDf['predict'], True, False)
    PercentageDf = PercentageDf.groupby(['pat','predict']).size().to_frame('count').reset_index() # Count the True and False grouped by patients
    
    
    PercentageDf = PercentageDf.pivot(index='pat', columns='predict', values='count') # Creeer een pivot tabel waar de true and false naar colommen gezet worden ipv twee rijen per patient
    PercentageDf.columns = ['countF', 'countT'] # Hernoem de colommen naar CountF en CountT
    PercentageDf['percentage'] = np.nan_to_num(100 * PercentageDf['countT'] / (PercentageDf['countT'] + PercentageDf['countF'])) # Bereken het percentage dat goed geraden is
#     print(PercentageDf)
#     ## maak tabel (df4) met per patient het aantal sampels dat true en false gelabeld wordt.

#     # aantal false tellen
    
#     df2 = df_cleaned.groupby(['pat','predict']).size().to_frame('countF').reset_index()
#     df2 = df2.where(df2['predict'] == False).dropna()
#     df2 = df2.set_index('pat')

#     # aantal true tellen
#     df3 = df_cleaned.groupby(['pat','predict']).size().to_frame('countT').reset_index()
#     df3 = df3.where(df3['predict'] == True).dropna()
#     df3 = df3.set_index('pat')

#     # join df2 en df3
#     df4 = pd.concat([df2, df3], axis=1)
#     df4.drop(['predict','predict'], axis=1, inplace=True)
#     df4.fillna(0, inplace=True)

#     # bereken percentage
#     df4['percentage'] = 100 * df4['countT'] / (df4['countT'] + df4['countF'])
#     df4.sort_values(by=['percentage'],ascending=False).head(30)

    Data[class_2b_found]['result'] = PercentageDf
    # <-- End Percentage check -->

Iteration 1, loss = 0.23728238
Iteration 2, loss = 0.12182436
Iteration 3, loss = 0.08001435
Iteration 4, loss = 0.05787191
Iteration 5, loss = 0.04231146
Iteration 6, loss = 0.03256677
Iteration 7, loss = 0.02517756
Iteration 8, loss = 0.02251306
Iteration 9, loss = 0.02022401
Iteration 10, loss = 0.01769724
Iteration 11, loss = 0.01420624
Iteration 12, loss = 0.01074548
Iteration 13, loss = 0.01569606
Iteration 14, loss = 0.01001124
Iteration 15, loss = 0.01104224
Iteration 16, loss = 0.01268172
Iteration 17, loss = 0.00688000
Iteration 18, loss = 0.00869505
Iteration 19, loss = 0.00715325
Iteration 20, loss = 0.00966980
Iteration 21, loss = 0.00770285
Iteration 22, loss = 0.00700489
Iteration 23, loss = 0.00646064
Iteration 24, loss = 0.00790017
Iteration 25, loss = 0.00689971
Iteration 26, loss = 0.00571669
Iteration 27, loss = 0.00340460
Iteration 28, loss = 0.00918025
Iteration 29, loss = 0.00772865
Iteration 30, loss = 0.00310122
Iteration 31, loss = 0.00828089
Iteration 32, los

Iteration 94, loss = 0.01213293
Iteration 95, loss = 0.01333766
Iteration 96, loss = 0.01148712
Iteration 97, loss = 0.00834240
Iteration 98, loss = 0.01313864
Iteration 99, loss = 0.00964470
Iteration 100, loss = 0.01178376
Training loss did not improve more than tol=0.000010 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.08308013
Iteration 2, loss = 0.01936690
Iteration 3, loss = 0.01065739
Iteration 4, loss = 0.00635407
Iteration 5, loss = 0.00501570
Iteration 6, loss = 0.00374457
Iteration 7, loss = 0.00354896
Iteration 8, loss = 0.00338544
Iteration 9, loss = 0.00262755
Iteration 10, loss = 0.00284587
Iteration 11, loss = 0.00205317
Iteration 12, loss = 0.00245188
Iteration 13, loss = 0.00214339
Iteration 14, loss = 0.00153068
Iteration 15, loss = 0.00044087
Iteration 16, loss = 0.00182407
Iteration 17, loss = 0.00386028
Iteration 18, loss = 0.00043740
Iteration 19, loss = 0.00179222
Iteration 20, loss = 0.00059806
Iteration 21, loss = 0.00148980
Iteration 22, loss = 0

# Visualize it

In [6]:
# PlotData(5,17)

# Information matrix

In [7]:
for cat in classes:
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html
    y_true = Data[cat]['y_test']
    y_pred = Data[cat]['Model'].predict(Data[class_2b_found]['X_test'])

    TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()

    # tabel printen

    print('Informationi matrix %s' % cat)
    tab = [["pred pos", TP, FP], ["pred neg", FN, TN]]
    print(pd.DataFrame(tab, columns=["", "pos", "neg"]))
    print()
    print("recall: ", recall_score(y_true, y_pred))
    print("precision: ", precision_score(y_true, y_pred))
    print("accuracy: ", accuracy_score(y_true, y_pred))
    print('\n\n')

Informationi matrix c1
              pos    neg
0  pred pos  6722      8
1  pred neg    10  27549

recall:  0.9985145573380867
precision:  0.9988112927191679
accuracy:  0.9994750503076788



Informationi matrix c2
               pos    neg
0  pred pos  13650    138
1  pred neg     59  20442

recall:  0.995696257932745
precision:  0.9899912967798086
accuracy:  0.9942547172562629



Informationi matrix c3
               pos    neg
0  pred pos  11688     98
1  pred neg     79  22424

recall:  0.993286309169712
precision:  0.9916850500593924
accuracy:  0.9948379946921754



Informationi matrix c4
              pos    neg
0  pred pos  2080      2
1  pred neg     1  32206

recall:  0.9995194617972128
precision:  0.9990393852065321
accuracy:  0.9999125083846131





# Analyse Percentages

In [8]:
#collumn namen renamen
for num, i in enumerate(classes):
    Data[i]['result'].rename(columns={ Data[i]['result'].columns[2]: "percentagecat%s" % str(num+1) },inplace=True)


#alle resultaten in 1 df joinen
resultdf5 = pd.concat([Data['c1']['result'], \
                       Data['c2']['result'], \
                       Data['c3']['result'], \
                       Data['c4']['result']], axis=1, join='inner')

# Voorbereiden datafarme, creeer het antwoord (groundtruth) en een bias
resultdf5.reset_index(inplace=True)
resultdf5['groundtruth'] = resultdf5['pat'].apply(CategorieStringToNum)
resultdf5['bias'] = 1
resultdf5.set_index('pat', drop=True, inplace=True)

param = ['bias','percentagecat1', 'percentagecat2', 'percentagecat3', 'percentagecat4']
yValueColumn = 'groundtruth'    
X_param = np.matrix(resultdf5[param])
y = resultdf5['groundtruth']

# Use SKLearn.linear_model
model_param = MLPClassifier(hidden_layer_sizes=(50,40,50),solver='adam',random_state=21,tol=0.00001,learning_rate='adaptive',verbose=10)
model_param.fit(X_param, y)
y_pred = model_param.predict(X_param)

# kolom met voorspelling toevoegen aan data)
resultdf5['predict'] = y_pred
try:
    resultdf5['predict_round'] = [int(round(i)) if i <4 and i > 1 else 4 if i > 4 else 1 for i in y_pred]
except:
    resultdf5['predict_round'] = list(resultdf5['predict'])
resultdf5 = resultdf5.drop(['countF', 'countT', ], axis=1)
resultdf5.reset_index(inplace=True)

## maak tabel (df4) met per patient het aantal sampels dat true en false gelabeld wordt.

# aantal false tellen
df2 = resultdf5.groupby(['pat','predict_round']).size().to_frame('countF').reset_index()
df2.set_index(['pat', 'predict_round'], inplace=True)
df2 = df2.unstack(-1)
df2 = df2.join(resultdf5[['pat',yValueColumn]].drop_duplicates(subset=['pat', yValueColumn]).set_index('pat'))


df2['percentage'] = np.vectorize(CalculatePerc)(df2[yValueColumn], df2['countF', 1],df2['countF', 2],df2['countF', 3],df2['countF', 4])
df2.sort_values(by=['percentage'],ascending=False)
print(df2)

Iteration 1, loss = 6.40077847
Iteration 2, loss = 5.05010951
Iteration 3, loss = 4.11844776
Iteration 4, loss = 3.26602346
Iteration 5, loss = 2.45296633
Iteration 6, loss = 1.71083383
Iteration 7, loss = 1.18318715
Iteration 8, loss = 0.91092100
Iteration 9, loss = 0.77768013
Iteration 10, loss = 0.71049456
Iteration 11, loss = 0.68381627
Iteration 12, loss = 0.67238459
Iteration 13, loss = 0.66640138
Iteration 14, loss = 0.66241314
Iteration 15, loss = 0.65934353
Iteration 16, loss = 0.65686348
Iteration 17, loss = 0.65486556
Iteration 18, loss = 0.65307726
Iteration 19, loss = 0.65143676
Iteration 20, loss = 0.64992387
Iteration 21, loss = 0.64851394
Iteration 22, loss = 0.64718658
Iteration 23, loss = 0.64594465
Iteration 24, loss = 0.64476350
Iteration 25, loss = 0.64365106
Iteration 26, loss = 0.64258780
Iteration 27, loss = 0.64156033
Iteration 28, loss = 0.64056369
Iteration 29, loss = 0.63956355
Iteration 30, loss = 0.63858744
Iteration 31, loss = 0.63763950
Iteration 32, los

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


KeyError: ('countF', 1)

In [ ]:
resultdf5['predict']

In [ ]:
print("Accurracy: " + str(round((df2.percentage.sum()/100)/df2.percentage.count()*100,2)) + "%")


In [ ]:
resultdf5 = pd.concat([Data['c1']['result'], \
                       Data['c2']['result'], \
                       Data['c3']['result'], \
                       Data['c4']['result']], axis=1, join='inner')

# Voorbereiden datafarme, creeer het antwoord (groundtruth) en een bias
resultdf5.reset_index(inplace=True)
resultdf5['groundtruth'] = resultdf5['pat'].apply(CategorieStringToNum)
resultdf5['bias'] = 1
resultdf5.set_index('pat', drop=True, inplace=True)

resultdf5[['percentagecat1', 'percentagecat2', 'percentagecat3', 'percentagecat4']]

In [ ]:
datatemp = df_cleaned.loc[df_cleaned["pat"]==1002]


for i in classes:
    print(i)
    ypredict = Data[i]['Model'].predict_proba(datatemp[Xcolumns])
    for x in ypredict[0]:
        print(format(x, ".20f"))
    


# Things for later
## Check most importent parameter

    ParamCheck = pd.DataFrame()
    ParamCheck['Params'] = list(df_cleaned[Xcolumns])
    ParamCheck['Values'] = list(lr.coef_)[0]
    ParamCheck['Absolute'] = np.absolute(list(ParamCheck['Values']))
    ParamCheck = ParamCheck.sort_values(by=['Absolute'],ascending=False)

## Check if plot_decision_boundary is correct
    plot_decision_boundary werkt nog via de oude manier, niet via sklearn, en gebruikt misschien niet dezelfdemanier als het model?
    Voor nu is dat misschien nog oke, maar als we geen lineare regressie meer gaan doen niet meer.